In [1]:
from dataclasses import dataclass
from typing import Optional, Tuple
import math

@dataclass
class PatientData:
    # Demographics
    question2: int  # Age
    question3: str  # Gender
    question6: Optional[float] = None  # Height in cm
    question7: Optional[float] = None  # Weight in kg
    
    # Risk Factors
    question4: Optional[str] = None  # Current smoker status
    question5: Optional[str] = None  # Diabetes status
    question65: Optional[str] = None  # Family history of early heart disease
    question24: Optional[str] = None  # Using blood pressure medication
    question25: Optional[float] = None  # Systolic blood pressure
    question8: Optional[str] = None  # History of cardiovascular disease
    question9: Optional[str] = None  # PCOS history
    question10: Optional[str] = None  # Gestational diabetes history
    question11: Optional[str] = None  # HIV status
    question12: Optional[str] = None  # Family history of diabetes
    question13: Optional[str] = None  # Knows A1C value
    question14: Optional[float] = None  # A1C value if known

def calculate_bmi(height_cm: float, weight_kg: float) -> float:
    """Calculate BMI given height in cm and weight in kg."""
    height_m = height_cm / 100
    return weight_kg / (height_m * height_m)

def analyze_screening(patient_data: PatientData) -> Tuple[str, str, str]:
    """
    Analyzes whether a patient should undergo lipid, Lp(a), and diabetes screening.
    Returns recommendations for hyperlipidemia, Lp(a), and diabetes screening.
    """
    lipid_recommendation = analyze_hyperlipidemia_screening(patient_data)
    lpa_recommendation = analyze_lpa_screening(patient_data)
    diabetes_recommendation = analyze_diabetes_screening(patient_data)
    
    return lipid_recommendation, lpa_recommendation, diabetes_recommendation

def analyze_lpa_screening(patient_data: PatientData) -> str:
    """
    Analyzes whether Lp(a) screening is indicated based on guidelines.
    Focuses on demographic and historical risk factors only.
    """
    age = patient_data.question2
    gender = "female" if patient_data.question3 == "Item 1" else "male"
    
    # Check for high-risk conditions that warrant earlier screening
    risk_factors = []
    
    # Family history of premature ASCVD
    if patient_data.question65 == "Item 2":
        risk_factors.append("family history of premature cardiovascular disease")
    
    # Personal history of cardiovascular disease
    if patient_data.question8 == "Item 2":
        risk_factors.append("personal history of cardiovascular disease")
    
    # Check for additional cardiovascular risk factors
    if patient_data.question4 == "Item 2":  # Current smoker
        risk_factors.append("current smoker")
    
    if patient_data.question5 == "Item 2":  # Has diabetes
        risk_factors.append("diabetes")
    
    if patient_data.question24 == "Item 1":  # On blood pressure medication
        risk_factors.append("hypertension on medication")
    elif patient_data.question25 and patient_data.question25 >= 130:  # Elevated BP
        risk_factors.append("elevated blood pressure")
    
    # Determine recommendation
    if age < 20:
        return "Lp(a) screening not routinely indicated before age 20."
    
    # Early screening if risk factors present and age ≥20
    if (age >= 20 and risk_factors) or age >= 30:
        base_recommendation = (
            "Lp(a) screening is indicated. "
        )
        
        if risk_factors:
            base_recommendation = (
                f"Lp(a) screening is strongly indicated due to presence of: "
                f"{', '.join(risk_factors)}. " + base_recommendation
            )
        
        return base_recommendation
    
    return (
        "Consider Lp(a) screening if cardiovascular risk factors develop. "
        "Routine screening indicated starting at age 30."
    )

def analyze_hyperlipidemia_screening(patient_data: PatientData) -> str:
    """
    Analyzes whether a patient should undergo hyperlipidemia screening based on guidelines.
    """
    age = patient_data.question2
    
    # Check if they're in the adolescent screening range (17-21)
    if 17 <= age <= 21:
        return "Indicated for one-time screening with non-fasting non-HDL cholesterol test."
    
    # Determine risk factors
    risk_factors = []
    
    if patient_data.question4 == "Item 2":  # Current smoker
        risk_factors.append("smoking")
    
    if patient_data.question5 == "Item 2":  # Has diabetes
        risk_factors.append("diabetes")
        
    if patient_data.question65 == "Item 2":  # Family history of early heart disease
        risk_factors.append("family history of early heart disease")
        
    if patient_data.question8 == "Item 2":  # History of cardiovascular disease
        risk_factors.append("history of cardiovascular disease")
        
    # Check for hypertension
    if patient_data.question24 == "Item 1":  # Using blood pressure medication
        risk_factors.append("hypertension (on medication)")
    elif patient_data.question25 and patient_data.question25 >= 130:  # Systolic BP ≥ 130
        risk_factors.append("hypertension (elevated blood pressure)")
    
    is_high_risk = len(risk_factors) > 1  # High risk if multiple risk factors
    
    # Determine screening recommendation based on age, gender, and risk status
    gender = "female" if patient_data.question3 == "Item 1" else "male"
    
    if is_high_risk:
        if (gender == "male" and age >= 25) or (gender == "female" and age >= 35):
            recommendation = (
                f"Indicated for lipid screening due to high risk status "
                f"(multiple risk factors: {', '.join(risk_factors)}). "
                "Recommend measuring total cholesterol and HDL cholesterol."
            )
        else:
            recommendation = "Not yet indicated for screening based on age, despite high risk status. Continue monitoring risk factors."
    else:
        if (gender == "male" and age >= 35) or (gender == "female" and age >= 45):
            recommendation = (
                "Indicated for routine lipid screening based on age. "
                "Recommend measuring total cholesterol and HDL cholesterol."
            )
        else:
            recommendation = "Not yet indicated for routine screening based on age and risk factors."
    
    # Add note about frequency if already being screened
    if age < 65 and "Indicated" in recommendation:
        recommendation += " Repeat screening every 5 years if results are below treatment threshold."
    elif age >= 65:
        recommendation += " Consider stopping routine screening if multiple previous lipid screenings were normal."
    
    return recommendation

def analyze_diabetes_screening(patient_data: PatientData) -> str:
    """
    Analyzes whether diabetes screening is indicated based on ADA guidelines.
    """
    age = patient_data.question2
    risk_factors = []
    high_risk_conditions = []

    # Check BMI if height and weight are provided
    if patient_data.question6 and patient_data.question7:
        bmi = calculate_bmi(patient_data.question6, patient_data.question7)
        if bmi >= 25:  # Simplified threshold, could be adjusted for Asian Americans
            risk_factors.append("elevated BMI")

    # Check for gestational diabetes history
    if patient_data.question10 == "Item 2":  # Has gestational diabetes history
        high_risk_conditions.append("history of gestational diabetes")

    # Check for PCOS
    if patient_data.question9 == "Item 2":  # Has PCOS
        risk_factors.append("polycystic ovary syndrome (PCOS)")

    # Check for HIV
    if patient_data.question11 == "Item 2":  # Has HIV
        high_risk_conditions.append("HIV positive status")

    # Check for family history of diabetes
    if patient_data.question12 == "Item 2":  # Has family history
        risk_factors.append("first-degree relative with diabetes")

    # Check for hypertension
    if patient_data.question24 == "Item 1":  # On blood pressure medication
        risk_factors.append("hypertension (on medication)")
    elif patient_data.question25 and patient_data.question25 >= 130:
        risk_factors.append("elevated blood pressure")

    # Check for cardiovascular disease
    if patient_data.question8 == "Item 2":
        risk_factors.append("history of cardiovascular disease")

    # Check A1C if known
    prediabetes = False
    if patient_data.question13 == "Item 2" and patient_data.question14:
        if patient_data.question14 >= 5.7:
            prediabetes = True
            high_risk_conditions.append("prediabetes (A1C ≥5.7%)")

    # Determine screening recommendation
    if age < 35 and not (risk_factors or high_risk_conditions):
        return "Routine diabetes screening not indicated before age 35 unless risk factors develop."

    if high_risk_conditions:
        conditions = ", ".join(high_risk_conditions)
        if "history of gestational diabetes" in conditions:
            return f"Diabetes screening strongly indicated due to {conditions}. Recommend lifelong testing at least every 3 years."
        elif "prediabetes" in conditions:
            return f"Diabetes screening strongly indicated due to {conditions}. Recommend yearly testing."
        else:
            return f"Diabetes screening strongly indicated due to {conditions}. Recommend regular testing."

    if risk_factors:
        factors = ", ".join(risk_factors)
        return f"Diabetes screening indicated due to presence of risk factors: {factors}. Recommend testing every 3 years or more frequently based on results and risk status."

    if age >= 35:
        return "Routine diabetes screening indicated based on age. Recommend testing every 3 years if results are normal."

    return "Consider diabetes screening if additional risk factors develop."

if __name__ == "__main__":
    # Example usage
    patient = PatientData(
        question2=45,  # Age
        question3="Item 1",  # Female
        question4="Item 2",  # Current smoker
        question5="Item 2",  # Has diabetes
        question65="Item 2",  # Family history present
        question24="Item 1",  # On blood pressure medication
        question25=135,  # Systolic blood pressure
        question8="Item 2",  # Has cardiovascular disease
        question6=170,  # Height in cm
        question7=75,  # Weight in kg
        question9="Item 2",  # Has PCOS
        question10="Item 2",  # Has gestational diabetes history
        question11="Item 1",  # No HIV
        question12="Item 2",  # Family history of diabetes
        question13="Item 2",  # Knows A1C
        question14=5.8  # A1C value
    )
    
    lipid_rec, lpa_rec, diabetes_rec = analyze_screening(patient)
    print("\nHyperlipidemia Screening Recommendation:")
    print(lipid_rec)
    print("\nLp(a) Screening Recommendation:")
    print(lpa_rec)
    print("\nDiabetes Screening Recommendation:")
    print(diabetes_rec)


Hyperlipidemia Screening Recommendation:
Indicated for lipid screening due to high risk status (multiple risk factors: smoking, diabetes, family history of early heart disease, history of cardiovascular disease, hypertension (on medication)). Recommend measuring total cholesterol and HDL cholesterol. Repeat screening every 5 years if results are below treatment threshold.

Lp(a) Screening Recommendation:
Lp(a) screening is strongly indicated due to presence of: family history of premature cardiovascular disease, personal history of cardiovascular disease, current smoker, diabetes, hypertension on medication. Lp(a) screening is indicated. 

Diabetes Screening Recommendation:
Diabetes screening strongly indicated due to history of gestational diabetes, prediabetes (A1C ≥5.7%). Recommend lifelong testing at least every 3 years.
